### 用Tools Call來做一個旅游規劃對話機器人

**背景：** 在現代忙碌的社會中，每天都在期待假期的到來，想著一定要去哪裏哪裏玩，一定要出國等等。但往往在假期真正到來的時候，我們卻因爲懶得規劃交通、住宿、旅游成本等，或無法做出完美的規劃而選擇窩在家裏。今天設計的這個對話機器人最主要的區別在於它擅長處理使用者模糊不確定的要求，再結合外部API來得到航班、住宿、景點等資訊，來提高使用者規劃旅程的意願。

**靈感來源：** 之前使用爬蟲程式做過相關的特定地點和日期下航班價格和酒店價格之間的關係，瞭解在有大量資料的情況下，能做出許多應用的延申。考慮到使用效率，現在改用API來替代我的爬蟲程式。

**目標：** 特定對話情境下，LLM能理解使用者的要求，並選擇正確的工具（Tools Call）
****

**使用情景：** 主要提供給想法還不夠明確的使用者，比如：

- **日期不確定：** “我剛好某月中有7天連續假期，但我只想規劃4天的旅程。”
- **成本考量：** “我想在這個時段找到最優惠的價格（機票、住宿），但我懶得去一一比較”

****

**API： Amadeus for Developer**
- 提供self-service API（可透過API請求測試環境下的資料，驗證程式軟體的流程和正確性）
- 此專案僅使用測試環境下的API請求

**工具：**
- search_best_flight_offer
    - 獲取範圍日期以内最優惠的航班資訊
- extract_flight_parameters
    - 從模糊輸入中提取參數、補足參數
- search_best_hotel_offers
    - 獲取住宿資訊

**LLM：** OpenAI gpt-4o-mini

#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [ ]:
import os
from google.colab import userdata

全局狀態記憶體

In [ ]:
CONVERSATION_STATE = {
    "intent": None,           # 當前意圖 (例如: "flight_search")
    "params": {},             # 已收集的參數 (例如: {'origin': 'TPE'})
    "missing_slots": []       # 缺失的關鍵參數
}

# 航班搜尋工具的 JSON Schema 定義 (STEP 1)

In [ ]:
# --------------------------------------------------
# 航班搜尋工具的 JSON Schema 定義 (STEP 1)
# --------------------------------------------------
FLIGHT_SEARCH_TOOL_SCHEMA = {
    "type": "function",
    "function": {
        "name": "search_best_flight_offer",
        "description": "搜尋指定月份和日期範圍內，最便宜的往返航班報價。需要出發地、目的地、開始日期、結束日期和總天數。",
        "parameters": {
            "type": "object",
            "properties": {
                "origin": {"type": "string", "description": "出發機場/城市代碼 (TPE, NRT, TYO)"},
                "destination": {"type": "string", "description": "目的地城市代碼 (TYO, OSA, NYC)"},
                "start_date": {"type": "string", "description": "搜尋出發日的開始日期，格式 YYYY-MM-DD (例如 2026-06-05)"},
                "end_date": {"type": "string", "description": "搜尋回程日的結束日期，格式 YYYY-MM-DD (例如 2026-06-25)"},
                "duration": {"type": "integer", "description": "旅程的總天數 (例如 7 天)"},
                "adults": {"type": "integer", "description": "旅客人數，預設為 1"}
            },
            "required": ["origin", "destination", "start_date", "end_date", "duration"]
        }
    }
}

參數提取工具

In [ ]:
PARAM_EXTRACTOR_SCHEMA = {
    "type": "function",
    "function": {
        "name": "extract_flight_parameters",
        "description": "從用戶輸入中提取航班搜尋所需的所有參數。提取到的值請轉換為函式參數的要求格式。",
        "parameters": {
            "type": "object",
            "properties": {
                "origin": {"type": "string", "description": "出發機場/城市代碼 (TPE, NRT, TYO)。如果用戶沒有給出，則忽略。"},
                "destination": {"type": "string", "description": "目的地城市代碼 (TYO, OSA, NYC)。如果用戶沒有給出，則忽略。"},
                "start_date": {"type": "string", "description": "搜尋出發日的開始日期，格式 YYYY-MM-DD。請自行判斷年份 (假設為 2026)。如果用戶沒有給出，則忽略。"},
                "end_date": {"type": "string", "description": "搜尋回程日的結束日期，格式 YYYY-MM-DD。請自行判斷年份 (假設為 2026)。如果用戶沒有給出，則忽略。"},
                "duration": {"type": "integer", "description": "旅程的總天數。如果用戶沒有給出，則忽略。"},
            }
        }
    }
}

爲了穩定使用aisuite的tools call功能

這裏使用的LLM是OpenAi的gpt-4o-mini

In [ ]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('OpenAi')
os.environ['OPENAI_API_KEY']=api_key
provider_writer = "openai"
model_writer = "gpt-4o-mini"

In [ ]:
!pip install aisuite[all]

### 2. 使用 AISuite 的準備

In [ ]:
import aisuite as ai

In [ ]:
provider = "openai"
model = "gpt-4o-mini"

provider_planner = "openai"
model_planner = "gpt-4o-mini"

provider_writer = "openai"
model_writer = "gpt-4o-mini"

In [ ]:
# ... (您的其他 import) ...
# import aisuite as ai
# ...

def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="openai/gpt-oss-120b",
          tools=None,         # ❗ 新增參數 1: 傳入工具定義 ❗
          tool_choice="auto"  # ❗ 新增參數 2: 讓模型自動決定是否使用工具 ❗
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]

    response = client.chat.completions.create(
        model=f"{provider}:{model}",
        messages=messages,
        tools=tools,         # 傳遞工具定義
        tool_choice=tool_choice # 讓模型決定是否呼叫
    )

    # ❗ 檢查模型是否要求呼叫函式 ❗
    if response.choices[0].message.tool_calls:
        return response.choices[0].message.tool_calls

    return response.choices[0].message.content

# 航班搜索工具

Amadeus API


In [ ]:
client_id = userdata.get('AmadeusID')
client_secret = userdata.get('AmadeusSecret')

In [ ]:
!pip install amadeus

In [ ]:
from amadeus import Client

非同步循環API請求

In [ ]:
import nest_asyncio

工具method

In [ ]:
from typing import Optional
import json

# ----------------------------------------------------------------------
# 酒店搜尋工具 (Hotels Search Tool - 使用 Amadeus API - 強化 Log)
# ----------------------------------------------------------------------
def search_best_hotel_offers(city_code: str, check_in_date: str, check_out_date: str, guests: Optional[int] = 1) -> str:
    """
    使用 Amadeus API 查詢指定城市、入住/退房日期下的最優惠酒店報價。
    """
    print("\n" + "~"*50)
    print(f"🏨 TOOL CALL: search_best_hotel_offers 收到參數:")
    print(f"  City Code: {city_code}, Check-in: {check_in_date}, Check-out: {check_out_date}, Guests: {guests}")
    print("~"*50)

    # 儲存原始日期，用於結果輸出
    original_in_date = check_in_date
    original_out_date = check_out_date

# ❗ 核心修正：日期滾動校正邏輯 ❗
    try:
        now = datetime.now()

        in_dt = datetime.strptime(check_in_date, '%Y-%m-%d')
        out_dt = datetime.strptime(check_out_date, '%Y-%m-%d')

        # --- 處理入住日期 (in_dt) ---
        # 1. 將 in_dt 的年份設為今年
        in_dt_this_year = in_dt.replace(year=now.year)

        # 2. 如果 in_dt 已經在過去 (例如：今天是 12/15，但 in_dt 是 12/14)
        if in_dt_this_year <= now:
            # 必須推到明年，確保它落在未來
            in_dt = in_dt_this_year.replace(year=now.year + 1)
        else:
            # 否則就用今年的日期
            in_dt = in_dt_this_year

        # --- 處理退房日期 (out_dt) ---
        # 3. 將 out_dt 的年份設為與修正後的 in_dt 相同的年份
        out_dt = out_dt.replace(year=in_dt.year)

        # 4. 檢查退房日期是否在入住日期之前 (跨年問題)
        # 如果退房日期比修正後的入住日期還要早，則將退房日期的年份往後推一年。
        if out_dt <= in_dt:
            out_dt = out_dt.replace(year=in_dt.year + 1)


        # 更新函數內部的日期變數
        check_in_date = in_dt.strftime('%Y-%m-%d')
        check_out_date = out_dt.strftime('%Y-%m-%d')

        if check_in_date != original_in_date:
            print(f"[DEBUG: Date Corrector] 發現入住日期 ({original_in_date}) 在過去，已自動校正為 {check_in_date} (未來最近)")
        if check_out_date != original_out_date and check_out_date != check_in_date:
             print(f"[DEBUG: Date Corrector] 退房日期 ({original_out_date}) 已自動校正為 {check_out_date}")

    except Exception as e:
        print(f"[ERROR: Date Corrector] 日期校正失敗: {e}")
        return json.dumps({"status": "error", "message": f"日期格式或校正錯誤: {e}"})
    # ❗ 核心修正結束 ❗

    try:
        amadeus = get_amadeus_client()
    except Exception as e:
        return json.dumps({"status": "error", "message": f"無法初始化 Amadeus 客戶端: {str(e)}"})

    # 1. 驗證和標準化城市代碼
    normalized_city_code = city_code.strip().upper()
    if len(normalized_city_code) != 3:
        return json.dumps({"status": "failure", "message": f"請提供 Amadeus City Code (三字代碼)。LLM推導失敗: {city_code}。"})

    # 2. 獲取該城市的前 10 個酒店 ID
    print(f"[DEBUG: Hotel Search] 正在查詢城市代碼 {normalized_city_code} 的酒店 ID...") # 輸出提示
    try:
        hotel_list_response = amadeus.reference_data.locations.hotels.by_city.get(
            cityCode=normalized_city_code,
        )

        hotel_ids = [hotel['hotelId'] for hotel in hotel_list_response.data[:10]]

        if not hotel_ids:
            print(f"[DEBUG: Hotel Search] 未找到酒店 ID。") # 輸出提示
            return json.dumps({"status": "failure", "message": f"在 Amadeus 資料庫中未找到 {normalized_city_code} 的任何酒店 ID。"})

        print(f"[DEBUG: Hotel Search] 成功獲取 {len(hotel_ids)} 個酒店 ID: {hotel_ids}") # 輸出提示

    except Exception as e:
        print(f"[ERROR: Hotel ID Search] 獲取酒店 ID 失敗: {e}")
        return json.dumps({"status": "error", "message": f"獲取酒店 ID 失敗，可能是城市代碼 {normalized_city_code} 無效或 API 故障。"})

    # 3. 查詢酒店報價
    hotel_ids_str = ",".join(hotel_ids)

    print(f"[DEBUG: Hotel Search] 正在查詢 {len(hotel_ids)} 個酒店的報價...") # 輸出提示
    print(f"[DEBUG: Hotel Search] 日期範圍: {check_in_date} 至 {check_out_date}") # 輸出提示

    try:
        hotel_offers_response = amadeus.shopping.hotel_offers_search.get(
            hotelIds=hotel_ids_str,
            checkInDate=check_in_date,
            checkOutDate=check_out_date,
            adults=str(guests),
            view='FULL'
        )

        offers_data = hotel_offers_response.data
        # ❗ 修正點：新增 API 狀態碼成功提示 ❗
        if hotel_offers_response.status_code == 200:
            print(f"[SUCCESS: API] 🥳 酒店報價 API 成功返回 [200 OK] 狀態！")

        if not offers_data:
            return json.dumps({"status": "failure", "message": f"在 {check_in_date} 至 {check_out_date} 期間，未找到任何酒店報價。"})

        print(f"[DEBUG: Hotel Search] 成功獲取 {len(offers_data)} 個酒店報價。") # 輸出提示

    except Exception as e:
        print(f"[ERROR: Hotel Offer Search] 查詢酒店報價失敗: {e}")
        return json.dumps({"status": "error", "message": f"查詢酒店報價失敗，API 故障或日期無效。: {str(e)}"})

    # 4. 格式化輸出
    summarized_hotels = []

    top_n = 5
    for i, offer in enumerate(offers_data[:top_n]):
        # ... 格式化邏輯保持不變 ...
        hotel_name = offer.get('hotel', {}).get('name', 'N/A')
        price_total = offer.get('offers', [{}])[0].get('price', {}).get('total', 'N/A')
        currency = offer.get('offers', [{}])[0].get('price', {}).get('currency', 'TWD')

        summarized_hotels.append({
            "rank": i + 1,
            "hotel_name": hotel_name,
            "price": f"{price_total} {currency}",
            "check_in": check_in_date,
            "check_out": check_out_date
        })

    print(f"[DEBUG: Hotel Search] 完成格式化，返回前 {len(summarized_hotels)} 個報價摘要。") # 輸出提示

    return json.dumps({
        "status": "success",
        "search_details": {"city_code": normalized_city_code, "check_in": check_in_date},
        "top_hotels": summarized_hotels
    })

In [ ]:
from typing import Optional
import json

def _summarize_offer(offer: dict, rank: int, default_currency: str) -> dict:
    """
    從單一 Amadeus 航班報價 (Offer) 中提取關鍵摘要資訊。
    """
    # 提取價格和幣別
    price = offer.get('price', {}).get('total', 'N/A')
    currency = offer.get('price', {}).get('currency', default_currency)

    itineraries = offer.get('itineraries', [])
    carrier_code = 'N/A'
    departure_date = 'N/A'
    return_date = 'N/A'
    summary = "無詳細行程資訊"

    if itineraries:
        first_itinerary = itineraries[0]
        segments = first_itinerary.get('segments', [])

        # 假設航空公司是第一個航段的承運人 (需要依賴 Amadeus Dictionaries 才能轉換為全名)
        if segments:
            carrier_code = segments[0].get('carrierCode', 'N/A')

            # 提取去程和回程的日期 (簡化提取)
            # 去程日期 (第一個航段的起飛時間)
            departure_date = segments[0].get('departure', {}).get('at', 'N/A')[:10]

            # 回程日期 (假設是最後一個航段的抵達時間，但可能需要更精確邏輯)
            if len(itineraries) > 1:
                 return_segments = itineraries[1].get('segments', [])
                 if return_segments:
                     return_date = return_segments[-1].get('arrival', {}).get('at', 'N/A')[:10]
            else:
                 # 單程或只有一個往返行程的情況
                 return_date = segments[-1].get('arrival', {}).get('at', 'N/A')[:10]

            # 建立簡潔的摘要
            summary = f"去程: {departure_date}，回程: {return_date}，主要承運商: {carrier_code}"

    return {
        "rank": rank,
        "price": f"{price} {currency}",
        "carrier_code": carrier_code,
        "summary": summary
    }

In [ ]:
# ----------------------------------------------------------------------
# 依賴庫導入
# ----------------------------------------------------------------------
# 確保以下已導入: from amadeus import Client, from datetime import datetime, timedelta, import json, import asyncio
# 確保以下已安裝: !pip install nest_asyncio

import asyncio
import json
import traceback
from datetime import datetime, timedelta
from typing import Optional
# 假設 Amadeus Client 已被正確導入 (from amadeus import Client)
from google.colab import userdata # 確保 userdata 在這裡也可用

AMADEUS_CLIENT = None  # 確保在任何函數外部定義它
CONVERSATION_STATE = {'intent': None, 'params': {}}

try:
    import nest_asyncio
    # 修正 Colab/Jupyter 環境下的 asyncio.run() 錯誤
    nest_asyncio.apply()
except ImportError:
    print("Warning: nest_asyncio not found. Async functions might fail in non-standard environments.")
except Exception:
    pass

def get_amadeus_client():
    """惰性初始化 Amadeus 客戶端。"""
    global AMADEUS_CLIENT
    if AMADEUS_CLIENT is None:
        try:
            from amadeus import Client
            client_id = userdata.get('AmadeusID')
            client_secret = userdata.get('AmadeusSecret')
            if not client_id or not client_secret:
                raise ValueError("Amadeus API 金鑰未配置。")

            # 初始化 Client
            AMADEUS_CLIENT = Client(client_id=client_id, client_secret=client_secret, hostname='test')
            print("[Tool Runner] Amadeus Client initialized.")
        except Exception as e:
            # 這是 critical error，必須返回 None 或報錯
            print(f"[Tool Runner] Amadeus Client Initialization Error: {e}")
            raise RuntimeError(f"Amadeus Client Initialization failed: {e}")

    return AMADEUS_CLIENT

# ----------------------------------------------------------------------
# 輔助函數：日期組合構建 (V4.0 - 使用完整日期字串)
# ----------------------------------------------------------------------
def build_flexible_search_bodies_v4(origin, destination, start_date, end_date, duration, adults):
    """
    V4.0：根據完整的日期字串 (YYYY-MM-DD) 和總天數，構建獨立的請求體。


    """

    print(f"[DEBUG: DateBuilder] 原始輸入: Start='{start_date}' ({type(start_date).__name__}), End='{end_date}' ({type(end_date).__name__}), Duration='{duration}' ({type(duration).__name__})")

# 核心修正：檢查並強制轉換為字串
    if start_date is None or end_date is None:
        print("[ERROR: DateBuilder] 缺少開始或結束日期。")
        return []

    try:
        start_date_str = str(start_date)
        end_date_str = str(end_date)

        # 進行日期解析
        start_dt = datetime.strptime(start_date_str, '%Y-%m-%d')
        end_dt = datetime.strptime(end_date_str, '%Y-%m-%d')
        trip_duration = int(duration)

    except ValueError as e:
        print(f"[ERROR: DateBuilder] 日期格式錯誤或 Duration 無效。錯誤: {e}")
        return []
    except Exception as e:
        print(f"[ERROR: DateBuilder] 類型轉換失敗。錯誤: {e}")
        return []

    # 設置參數
    current_date = start_dt
    search_limit_date = end_dt
    query_count = 0
    max_queries = 5
    individual_bodies = []

    # 核心邏輯：在 start_date 和 end_date 範圍內，每隔 1 天抽樣一次
    # ❗ 修正點 1：移除 'current_date.month == month' 檢查，只檢查日期範圍 ❗
    while current_date <= search_limit_date and query_count < max_queries:

        departure_date = current_date
        return_date = departure_date + timedelta(days=trip_duration)

        # 檢查回程日期是否在最終截止日期內
        # ❗ 修正點 2：移除 return_date.month == month 檢查 ❗
        if return_date <= search_limit_date.replace(hour=23, minute=59, second=59):

            single_body = {
                "currencyCode": "TWD",
                "travelers": [{"id": str(i+1), "travelerType": "ADULT"} for i in range(adults)],
                "sources": ["GDS"],
                "searchCriteria": {"maxFlightOffers": 20},
                "originDestinations": [
                    {
                        "id": query_count + 1,
                        "originLocationCode": origin,
                        "destinationLocationCode": destination,
                        "departureDateTimeRange": {"date": departure_date.strftime('%Y-%m-%d')},
                        "returnDateTimeRange": {"date": return_date.strftime('%Y-%m-%d')}
                    }
                ]
            }
            print(f"[DEBUG: DateBuilder] 成功構建查詢 {query_count+1}: {departure_date.strftime('%Y-%m-%d')} 至 {return_date.strftime('%Y-%m-%d')}")

            individual_bodies.append(single_body)
            query_count += 1
            current_date += timedelta(days=1)

        elif return_date > search_limit_date.replace(hour=23, minute=59, second=59):
            print("[DEBUG: DateBuilder] 回程日期超過範圍，停止構建。")
            break

        else:
             current_date += timedelta(days=1)

    print(f"[DEBUG: DateBuilder] 最終構建請求總數: {len(individual_bodies)}")
    return individual_bodies

# ----------------------------------------------------------------------
# 核心異步執行函數 (V4.0)
# ----------------------------------------------------------------------
async def _execute_flight_search_async_v4(amadeus_client, origin: str, destination: str, start_date: str, end_date: str, duration: int, adults: Optional[int] = 1):
    """
    V4.0 異步函數：循環發送 N 個單獨的 API 請求並找到最低價。
    """
    # # ❗ 修正點 3：修正參數名稱為 client_id ❗
    # try:
    #     from amadeus import Client
    #     amadeus = Client(client_id=client_id, client_secret=client_secret, hostname='test')
    # except Exception as e:
    #     print(f"[Tool Runner] Amadeus Client Initialization Error: {e}")
    #     return None

    print(f"[DEBUG: API Runner] 準備執行航班搜尋...")

    all_collected_offers = []

    # 1. 構建請求體列表 (使用 V4.0 函數)
    flight_bodies = build_flexible_search_bodies_v4(
        origin, destination, start_date, end_date, duration, adults
    )

    if not flight_bodies:
        print("[DEBUG: API Runner] 無請求體可執行，返回 None。")
        return None

    # lowest_price = float('inf')
    # best_offer = None

    for i, body in enumerate(flight_bodies):
        # ... API 請求和延遲邏輯保持不變 ...
        try:

            dep_date = body['originDestinations'][0]['departureDateTimeRange']['date']
            ret_date = body['originDestinations'][0]['returnDateTimeRange']['date']

            print(f"[DEBUG: API Runner] 發送請求 {i+1}/{len(flight_bodies)}: {body['originDestinations'][0]['departureDateTimeRange']['date']} -> {body['originDestinations'][0]['returnDateTimeRange']['date']}")

            flight_response = amadeus_client.shopping.flight_offers_search.post(body)
            data = flight_response.data

            print(f"[DEBUG: API Runner] 收到回應 {i+1}：狀態碼={flight_response.status_code}")

            if data:
                # ❗ 核心變更 2：將本次請求的所有結果加入總列表 ❗
                all_collected_offers.extend(data)
                print(f"[DEBUG: API Runner] 成功收集 {len(data)} 個航班報價。目前總數: {len(all_collected_offers)}")

            elif flight_response.status_code == 200 and not data:
                print(f"[DEBUG: API Runner] 收到 200 狀態，但無航班數據。")

        except Exception as e:
            # 捕獲 API 呼叫期間的錯誤，例如 400 錯誤等
            print(f"[ERROR: API Runner] 請求 {i+1} 發生 API 錯誤 ({origin}->{destination}, {start_date} -> {end_date}): {e}")
            pass # 繼續嘗試下一個查詢

        await asyncio.sleep(0.5)

    return all_collected_offers

# ----------------------------------------------------------------------
# LLM 呼叫的同步介面 (Function Calling Target - V4.0)
# ----------------------------------------------------------------------
def search_best_flight_offer(origin: str, destination: str, start_date: str, end_date: str, duration: int, adults: Optional[int] = 1) -> str:

    print("\n" + "~"*50)
    print(f"✈️ TOOL CALL: search_best_flight_offer 收到參數:")
    print(f"  Origin: {origin}, Destination: {destination}")
    print(f"  Dates: {start_date} -> {end_date}, Duration: {duration}, Adults: {adults}")
    print("~"*50)

    # 設置預設值，以防 LLM 傳入 None
    if adults is None: adults = 1

    # 儲存原始日期，用於結果輸出
    original_start_date = start_date
    original_end_date = end_date

    # 1. 日期校正 (保持您原有的邏輯)
    try:
        current_year = datetime.now().year

        # 解析 LLM 提供的日期
        llm_start_dt = datetime.strptime(start_date, '%Y-%m-%d')
        llm_end_dt = datetime.strptime(end_date, '%Y-%m-%d')

        if llm_start_dt.year <= current_year:
            correction_year = current_year + 1
            corrected_start_date = llm_start_dt.replace(year=correction_year).strftime('%Y-%m-%d')
            corrected_end_date = llm_end_dt.replace(year=correction_year).strftime('%Y-%m-%d')

            print(f"[DEBUG: Date Corrector] 發現查詢日期 ({start_date}) 在過去，已自動校正為 {corrected_start_date}")

            start_date = corrected_start_date # 更新為校正後的日期
            end_date = corrected_end_date     # 更新為校正後的日期

    except Exception as e:
        print(f"[ERROR: Date Corrector] 日期校正失敗: {e}")
        return json.dumps({"status": "error", "message": f"日期格式或校正錯誤: {e}"})


    # 2. 執行 V5 異步核心邏輯並獲取所有 Offers
    try:
        # 初始化客戶端
        amadeus = get_amadeus_client()

        print(f"\n[TOOL CALL] 啟動 V5 異步搜尋: {origin}->{destination}...")

        # ❗ 變更點 A: 接收所有 offers 的列表 ❗
        all_offers = asyncio.run(_execute_flight_search_async_v4(
            amadeus, origin, destination, start_date, end_date, duration, adults
        ))

    except RuntimeError as e:
        return json.dumps({"status": "error", "message": f"客戶端初始化錯誤或異步運行錯誤: {str(e)}"})
    except Exception as e:
        print(f"[Tool Runner] Critical Error during execution: {traceback.format_exc()}")
        return json.dumps({"status": "error", "message": f"工具執行發生無法修復的錯誤: {type(e).__name__}: {str(e)}"})

    # 3. 結果處理：排序、篩選和格式化
    if not all_offers:
        return json.dumps({
            "status": "failure",
            "message": f"在 {original_start_date} 到 {original_end_date} 期間，未找到從 {origin} 到 {destination} 的任何航班報價。"
        })

    try:
        # 排序：根據價格 (total) 進行升序排列
        def get_price_key(offer):
            try:
                return float(offer.get('price', {}).get('total', float('inf')))
            except (ValueError, TypeError):
                return float('inf')

        all_offers.sort(key=get_price_key)

        # 選取最優惠的前 N 個 (例如 N=5)
        top_n = 5
        top_offers = all_offers[:top_n]

        summarized_offers = []
        # 假設所有報價的幣別相同，取第一個作為預設
        default_currency = top_offers[0].get('price', {}).get('currency', 'TWD')

        for i, offer in enumerate(top_offers):
            # ❗ 變更點 B: 呼叫新的摘要函數 ❗
            summarized_offers.append(_summarize_offer(offer, i + 1, default_currency))

    except Exception as e:
        print(f"[ERROR: Offer Processing] 處理航班優惠時發生錯誤: {e}")
        return json.dumps({"status": "error", "message": "成功獲取數據，但在處理多個航班報價時發生錯誤。請檢查數據結構。"})

    # 4. 構建最終的 LLM 輸出
    final_result = {
        "status": "success",
        "message": f"成功找到從 {origin} 到 {destination} 在 {original_start_date} 到 {original_end_date} 期間的最優惠前 {len(summarized_offers)} 個航班報價。",
        "search_details": {
            "origin": origin,
            "destination": destination,
            "dates": f"{original_start_date} 至 {original_end_date}",
            "adults": adults,
            "duration": duration
        },
        "top_offers": summarized_offers # ❗ 變更點 C: 返回多個摘要結果 ❗
    }

    print(f"\n[FINAL RESULT] 準備返回 LLM 結果: {json.dumps(final_result, indent=2)}")

    return json.dumps(final_result)

# ----------------------------------------------------------------------
# 執行函式映射 (供 LLM 使用)
# ----------------------------------------------------------------------
# AVAILABLE_TOOLS = {
#     "search_best_flight_offer": search_best_flight_offer
# }

In [ ]:
def extract_flight_parameters(
    origin: Optional[str] = None,
    destination: Optional[str] = None,
    start_date: Optional[str] = None,
    end_date: Optional[str] = None,
    duration: Optional[int] = None
) -> str:
    # 這個函數只是一個介面，讓 LLM 輸出結構化的 JSON
    return json.dumps({
        "origin": origin,
        "destination": destination,
        "start_date": start_date,
        "end_date": end_date,
        "duration": duration
    })

####  3. 打造對話機器人

In [ ]:
system_writer = """
請用台灣習慣的中文回應: 你是一位專業的**旅遊規劃工作者**，專門負責為使用者查詢航班價格和酒店住宿，並根據結果提供最優惠的選擇和建議。

【一、工具使用原則】：
當使用者請求航班或酒店資訊時，請務必呼叫相應的工具。


【二、航班工具 (search_best_flight_offer) 規則】：
1. **[日期格式]**: 所有的日期參數都必須轉換為 **YYYY-MM-DD** 的完整格式。
2. **[參數推導與計算]**:
    a. 如果使用者給出一個他空閑的日期範圍(例如：6/5到6/12)，並給定行程天數，請將 6/5 設為 start_date，將 6/12 設為 end_date，請將行程天數設為 duration 參數。
    b. 如果使用者給出一個精確的往返日期 (例如：6/5到6/12)，請將 6/5 設為 start_date，將 6/12 設為 end_date。
    c. 如果使用者給出一個範圍 (例如：6月上半月，7天行程)，請將 6/1 設為 start_date，將 6/15 設為 end_date，duration 設為 7。
3. **[預設值]**: 如果 duration 參數缺失且無法從日期推導（例如只給了出發地），請使用預設值 **5** 天。
4. **[地點代碼 - 嚴格要求]**:
    **你必須**利用你的內部知識，將使用者輸入的所有城市或國家名稱（例如: 臺北, 日本, 西雅圖）轉換為有效的 **IATA 三字機場代碼** (例如: TPE, NRT, SEA)。
    **警告：** 絕對不能將中文或城市全名傳遞給 `origin` 或 `destination` 參數，否則 API 會失敗。

【三、酒店工具 (search_best_hotel_offers) 規則】：
5. **[必填代碼 - Amadeus City Code]**:
    當呼叫酒店工具時，`city_code` 參數必須是 **Amadeus City Code** (三字代碼，通常與 IATA 城市代碼相同，例如: **東京 -> TYO**，而不是 NRT)。
6. **[酒店日期]**: `check_in_date` 和 `check_out_date` 必須是 YYYY-MM-DD 格式。

【四、地點資訊查詢】：
6. 當使用者詢問地址、交通、或某個興趣點 (POI) 的詳細資訊時，請**優先使用 Google Search** 來提供即時、準確的資訊。
"""

In [ ]:
import aisuite as ai
import json

# ----------------------------------------------------------------------
# 核心控制函數：使用 GPT-4o mini 的自動工具執行模式 (最終修復版)
# ----------------------------------------------------------------------
def simplified_flight_response(user_prompt):
    """
    使用 aisuite 的 max_turns 模式，讓 GPT-4o mini 自動完成參數提取、
    狀態判斷、工具呼叫、結果處理和最終回覆。
    """
    global CONVERSATION_STATE

    # 1. 初始化 aisuite.Client
    try:
        client = ai.Client()
    except NameError:
        return "❌ 錯誤：aisuite 模組未正確導入。", "請確認您已執行 import aisuite as ai。"

    # 2. 準備提示詞 (這部分保持不變)
    current_params_str = json.dumps(CONVERSATION_STATE['params'])
    full_prompt = (f"這是目前的歷史記憶：{current_params_str}。使用者現在的請求是：'{user_prompt}'。 "
                   f"請利用可用的工具，執行參數提取或執行航班搜尋。請用繁體中文回覆。")

    # 3. 執行自動工具呼叫 (這部分保持不變)
    try:
        response = client.chat.completions.create(
            model=f"{provider_planner}:{model_planner}",
            messages=[
                {"role": "system", "content": system_writer},
                {"role": "user", "content": full_prompt}
            ],
            tools=[search_best_flight_offer, extract_flight_parameters, search_best_hotel_offers],
            max_turns=3
        )
    except Exception as e:
        return f"❌ LLM API 錯誤：{e}", "系統呼叫模型時發生錯誤，請檢查您的 API Key。"


    # 4. 處理回覆和中間訊息
    final_post = response.choices[0].message.content

    # ❗ 核心修正：穩健地將 Pydantic 物件轉換為字典，以避免 TypeError ❗
    serializable_messages = []

    for msg in response.choices[0].intermediate_messages:
        if hasattr(msg, 'model_dump'):
            # 情況 A: 它是 Pydantic/aisuite 物件
            try:
                # 嘗試將 Pydantic 物件轉換為字典
                serializable_messages.append(msg.model_dump())
            except Exception:
                # 失敗時，使用 model_dump_json 轉換為 JSON 字串，再轉回字典
                if hasattr(msg, 'model_dump_json'):
                    try:
                        json_str = msg.model_dump_json()
                        serializable_messages.append(json.loads(json_str))
                    except:
                         serializable_messages.append({"error": "Failed to serialize message fallback", "type": str(type(msg))})
                else:
                    serializable_messages.append({"error": "Failed to serialize message", "type": str(type(msg))})
        elif isinstance(msg, dict):
            # 情況 B: 它已經是字典
            serializable_messages.append(msg)
        else:
            # 情況 C: 其他無法序列化的物件
            serializable_messages.append({"error": "Non-dictionary/Pydantic object", "type": str(type(msg))})


    # 使用轉換後的列表進行安全的 JSON 序列化
    intermediate_messages_json = json.dumps(
        serializable_messages,
        indent=2,
        ensure_ascii=False
    )

    # 5. 更新狀態
    CONVERSATION_STATE['params'] = {}

    print("\n" + "="*80)
    print("✈️ LLM AGENT 最終執行 Log (GPT-4o mini Auto-Execution)")
    print("="*80)

    # 5.1. 檢查 LLM 是否決定呼叫了工具 (檢查第一個中間訊息)
    first_message = serializable_messages[0] if serializable_messages else None

    if first_message and first_message.get('tool_calls'):
        print("✅ LLM 決策：模型成功決定呼叫工具。")

        # 5.2. 檢查工具執行結果 (檢查後續的中間訊息)
        tool_results = [m for m in serializable_messages if m.get('role') == 'tool']

        if tool_results:
            print(f"✅ 工具呼叫總數：{len(tool_results)} 次執行。")

            for result in tool_results:
                tool_name = result.get('name', 'UNKNOWN')
                tool_content = result.get('content', '')

                # 嘗試解析工具內容 (Content) 來判斷狀態
                try:
                    content_data = json.loads(tool_content.strip('"')) # 需要 strip('"') 因為 content 被額外的引號包裹
                    status = content_data.get('status', 'PARSE_ERROR')

                    if status == "failure" or status == "error":
                        print(f"❌ 工具執行錯誤 [{tool_name}]：狀態={status}，訊息={content_data.get('message', '無訊息')}")
                    elif status == "success":
                        print(f"✅ 工具執行成功 [{tool_name}]：狀態=success。")
                    else:
                        print(f"⚠️ 工具執行警告 [{tool_name}]：未知狀態 {status}。")

                except (json.JSONDecodeError, AttributeError):
                    print(f"⚠️ 工具執行警告 [{tool_name}]：內容無法解析為標準 JSON。原始內容: {tool_content[:50]}...")
        else:
            print("⚠️ 警告：LLM 決定呼叫工具，但未返回工具結果（可能在第一回合結束）。")

    else:
        print("⚠️ LLM 決策：模型未決定呼叫工具，直接輸出文字回覆（可能在填槽階段）。")

    print("="*80)

    return intermediate_messages_json, final_post
# ----------------------------------------------------------------------

In [ ]:
AVAILABLE_TOOLS = {
    "search_best_flight_offer": search_best_flight_offer,
    "extract_flight_parameters": extract_flight_parameters, # 僅作為 LLM 的結構化輸出介面
    "search_best_hotel_offers": search_best_hotel_offers
}

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("### 🍀 窮游成本規劃機器人 🌈")
    gr.Markdown("請輸入你想要旅游的國家和日期！")
    user_input = gr.Textbox(label="關鍵詞：出發地、目的地、旅途日期、行程天數、住宿")
    btn = gr.Button("生成查詢結果")

    with gr.Row():
        # out1 將顯示：參數收集狀態 / 航班工具執行結果 JSON
        out1 = gr.Textbox(label="🧠 狀態/工具執行結果",
                          lines=20,
                          max_lines=100,
                          interactive=False)
        # out2 將顯示：最終 LLM 生成的回覆 (包括槽位填補提示或航班推薦)
        out2 = gr.Textbox(label="📣 回答",
                          lines=20,
                          max_lines=100,
                          interactive=False)

    # ❗ 修正點：將綁定函數從 lucky_post 更改為 lucky_post_with_state ❗
    # 由於 lucky_post_with_state 的返回值結構 (字典/JSON, 文字) 與原始的 (文字, 文字) 兼容，所以輸出元件不變。
    btn.click(simplified_flight_response, inputs=[user_input], outputs=[out1, out2])

In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e668dbbe1854d25ab3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
✈️ TOOL CALL: search_best_flight_offer 收到參數:
  Origin: TPE, Destination: NRT
  Dates: 2024-06-02 -> 2024-06-08, Duration: 4, Adults: 1
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[DEBUG: Date Corrector] 發現查詢日期 (2024-06-02) 在過去，已自動校正為 2026-06-02

[TOOL CALL] 啟動 V5 異步搜尋: TPE->NRT...
[DEBUG: API Runner] 準備執行航班搜尋...
[DEBUG: DateBuilder] 原始輸入: Start='2026-06-02' (str), End='2026-06-08' (str), Duration='4' (int)
[DEBUG: DateBuilder] 成功構建查詢 1: 2026-06-02 至 2026-06-06
[DEBUG: DateBuilder] 成功構建查詢 2: 2026-06-03 至 2026-06-07
[DEBUG: DateBuilder] 成功構建查詢 3: 2026-06-04 至 2026-06-08
[DEBUG: DateBuilder] 回程日期超過範圍，停止構建。
[DEBUG: DateBuilder] 最終構建請求總數: 3
[DEBUG: API Runner] 發送請求 1/3: 2026-06-02 -> 2026-06-06
[DEBUG: API Runner] 收到回應 1：狀態碼=200
[DEBUG: API Runner] 成功收集 20 個航班報價。目前總數: 20
[DEBUG: API Runner] 發送請求 2/3: 2026-06-03 -> 2026-06-07
[DEBUG: API Runner] 收到回應 2：狀態碼=200
[DEBUG: API Runner] 成功收集 20 個航班報價。目前總數: 40
[DEBUG: API Runner] 發送請求 3/3: 2